## Generate predictions and evaluate the model

### Introduction
I this notebook, we will run predictions on the model that we trained and deployed in the previous steps. If you recall, the model is hosted on Sagemaker realtime prediction endpoint. We will invoke that endpoint to generate the binary labels(1,0) on a few rows that we have in our test file (test_data.csv). We will then evaluate the results against the ground truth and see how the model performs.

### Prerequisits
Before proceeding, make sure you have run the following notebook in order without any errors:
1. variant_classifier-autopilot.ipynb
2. SageMakerAutopilotDataExplorationNotebook.ipynb
3. SageMakerAutopilotCandidateDefinitionNotebook.ipynb

If not, please go back to the notebooks and run them before proceeding.  

### Setup
Lets start by importing the libraries that we will need for executing this notebook.

In [ ]:
import pandas as pd
import sagemaker 
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV
import boto3
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Get the endpoint name
To generate predictions on test data, we need to get the endpoint name of the model that we deployed at the end of the SageMakerAutopilotCandidateDefinitionNotebook.ipynb notebook. To do this, we find the endpoint among the list of endpoints that starts with the string "AutoML-automl-vc". This is the default naming format that has been used in the variant_classifier-autopilot.ipynb and SageMakerAutopilotCandidateDefinitionNotebook.ipynb notebooks.

**NOTE:** If you changed the naming convention and/or have multiple endpoints beginning with the string "AutoML-automl-vc", the endpoint retrieved may not be the correct one. You can verify by logging into the AWS console, navigating to Sagemaker and selecting "Endpoints" from the left hand menu. Here you will see all the endpoints that have been created in your account. Select the one that you created as part of the SageMakerAutopilotCandidateDefinitionNotebook.ipynb notebook. If the correct endpoint is not selected, you can overwrite the name of the variable "endpoint_name" with the correct endpoint name. Make sure the correct endpoint is selected before proceeding.

In [ ]:
sm = boto3.client('sagemaker')
endpoints=sm.list_endpoints()['Endpoints']
for val in endpoints:
    ep=val.get("EndpointName")
    if ep.startswith('AutoML-automl-vc'):
        endpoint_name=ep
        print ('Model endpoint: '+endpoint_name)
        print ('Make sure this is the correct model endpoint before proceeding')
        break
    print('No endpoint found. Make sure you have completed the steps mentioned in the prerequisits above.')

### Data Preprocessing
We will now read the file "test_data.csv" into a dataframe and randomly sample 1000 records from it. 

In [ ]:
test_file=pd.read_csv('test_data.csv')
test_rows=test_file.sample(1000)
test_rows.head()

As you can see, the test rows look exactly like the rows in the training dataset as expected. We will now saperate out our target variable "CLASS" from the test data and store it in a new dataframe "actual"

In [ ]:
test_rows_notarget=test_rows.drop(['CLASS'],axis=1)
actual=test_rows['CLASS'].to_frame(name="actual")
actual.reset_index(drop=True, inplace=True)

### Generate Predictions
Next, we will invoke the endpoint of our model with the test rows and generate a prediction for each row. We will then store the results of the predciton in a new dataframe called "predicted".

In [ ]:
sm_session = sagemaker.Session()
variant_predictor=RealTimePredictor(endpoint=endpoint_name,sagemaker_session=sm_session,content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)

In [ ]:
predicted_str=variant_predictor.predict(test_rows_notarget.to_csv(sep=',', header=False, index=False)).decode('utf-8')
predicted=pd.Series(predicted_str.split(),name='predicted').to_frame().astype(int)

Finally, we combine "actual" and "predicted" values into a single dataframe called "results"

In [ ]:
results=pd.concat([actual, predicted],axis=1)
results.head()

### Model Evaluation
We will now generate some evaluation metrics for our binary classification model. We will start with a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) and follow that up with an [Receiver Operating Characteristic (ROC) curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic).

In [ ]:
cf_matrix = metrics.confusion_matrix(results['actual'], results['predicted'])
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues');

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(results['actual'], results['predicted'])

In [ ]:
roc_auc=metrics.auc(fpr, tpr)
accuracy=metrics.accuracy_score(results['actual'], results['predicted'])
precision=metrics.precision_score(results['actual'], results['predicted'])
recall=metrics.recall_score(results['actual'], results['predicted'])
f1score=metrics.f1_score(results['actual'], results['predicted'])
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.text(1.1,0.75,s='Accuracy: '+str(round(accuracy,2))+'\nPrecision: '+str(round(precision,2))+
'\nRecall: '+str(round(recall,2))+'\nF1 Score: '+str(round(f1score,2)),bbox=dict(boxstyle="square",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   ))

plt.show()